## Pre-trained models from keras

### Libraries

In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split

In [2]:
import cv2

from keras.utils import load_img, img_to_array, to_categorical
# from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from keras import backend as K
from keras import layers

from keras.models import Model, model_from_json

from keras.layers import Dropout, Dense, GlobalAveragePooling2D

# ResNet50
# from keras.applications.resnet50 import ResNet50, preprocess_input as resenet50_preprocess_input
from keras.applications.resnet import ResNet50, preprocess_input as resenet50_preprocess_input

# VGG16
from keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input

# InceptionV3
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input


from keras.optimizers import SGD, RMSprop, Adam

from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
K.set_image_data_format('channels_last')

### Loading

In [4]:
df = pd.read_csv('path/to/DDR/DR_grading/labels.csv')

# Define target sample sizes for each label
target_sizes = {
    0: 2000,
    1: 500,
    2: 500,
    3: 500,
    4: 500
}

# Initialize a list to hold the sliced DataFrames
sliced_dfs = []

# Slice the DataFrame for each label
for label, size in target_sizes.items():
    class_df = df[df['label'] == label]
    if len(class_df) >= size:
        # Undersample if the class size is greater than or equal to the target size
        sliced_df = class_df.sample(size, random_state=42)
    else:
        # Oversample if the class size is smaller than the target size
        sliced_df = resample(class_df, replace=True, n_samples=size, random_state=42)
    sliced_dfs.append(sliced_df)

# Combine all sliced DataFrames
final_df = pd.concat(sliced_dfs)

# Shuffle the final dataset
final_df = shuffle(final_df, random_state=42)

final_df['label'].value_counts()

label
0    2000
3     500
4     500
2     500
1     500
Name: count, dtype: int64

In [5]:
def enhance_img(img):
    # Step 1: Apply median filter with a 3x3 kernel
    img = cv2.medianBlur(img.astype(np.uint8), ksize=3)

    # Step 2: Convert to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab_img)

    # Step 3: Apply CLAHE on the Luminosity (L) channel with 8x8 tile grid
    clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8, 8))
    l = clahe.apply(l)

    # Step 4: Merge CLAHE enhanced L with original A and B channels
    lab_img = cv2.merge((l, a, b))

    # Step 5: Convert back to RGB color space
    enhanced_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2RGB)
    
    return enhanced_img


image_dir = 'path/to/DDR/DR_grading/all'

X = []
y = []

for idx, row in tqdm(final_df.iterrows()):
    
    image_path = row['image']
    label = row['label']
    
    image_path = os.path.join(image_dir, image_path)
    img = load_img(image_path, target_size=(224, 224))
    x = img_to_array(img)
    x = enhance_img(x)
    
    X.append(x)
    y.append(label)

# train test split
X = np.array(X)
y = np.array(y)

X.shape, y.shape

4000it [01:11, 56.32it/s]


((4000, 224, 224, 3), (4000,))

In [6]:
y_present = (y > 0).astype(int)  # Binary: 0 (no disease), 1 (disease present)
y_grades = np.where(y_present == 1, y, 0)  # Multiclass: 1-4 if disease present, 0 otherwise
y_grades = to_categorical(y_grades, num_classes=5)

X_train, X_test, y_present_train, y_present_test, y_grades_train, y_grades_test = train_test_split(
    X, y_present, y_grades, test_size=0.2, random_state=42)

X_train, X_val, y_present_train, y_present_val, y_grades_train, y_grades_val = train_test_split(
    X_train, y_present_train, y_grades_train, test_size=0.1, random_state=42)

### Load and preprocess according to pre-trianed model (ResNet50, ImageNet weights)

In [7]:
# Preprocess the input
X_train = resenet50_preprocess_input(X_train)
X_val = resenet50_preprocess_input(X_val)
X_test = resenet50_preprocess_input(X_test)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-12-12 09:57:40.576067: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-12 09:57:40.576922: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Add final layers

In [8]:
# last fully connected layer of resnet model
last_pretrained_layer = GlobalAveragePooling2D()(base_model.output)

fc_1 = Dense(1024, activation='relu')(last_pretrained_layer)
dropout_1 = Dropout(0.3)(fc_1)

fc_2 = Dense(512, activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(fc_2)

fc_3 = Dense(256, activation='relu')(dropout_2)
dropout_3 = Dropout(0.3)(fc_3)

fc_4 = Dense(128, activation='relu')(dropout_3)
dropout_4 = Dropout(0.3)(fc_4)

# Output for binary classification (presence of disease)
present_output = Dense(1, activation='sigmoid', name='present_output')(dropout_4)

# Output for multiclass grading (if disease present)
grading_fc_1 = Dense(64, activation='relu')(dropout_4)
grading_dropout_1 = Dropout(0.3)(grading_fc_1)

grading_fc_2 = Dense(32, activation='relu')(grading_dropout_1)
grading_dropout_2 = Dropout(0.3)(grading_fc_2)

grading_fc_3 = Dense(16, activation='relu')(grading_dropout_2)
grading_dropout_3 = Dropout(0.3)(grading_fc_3)

grading_output = Dense(5, activation='softmax', name='grading_output')(grading_dropout_3)

# Create the model
model = Model(inputs=base_model.input, outputs=[present_output, grading_output])


# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
losses = {
    'present_output': 'binary_crossentropy',
    'grading_output': 'categorical_crossentropy'
}
loss_weights = {
    'present_output': 1.0,  # Weight for disease presence
    'grading_output': 1.0  # Weight for disease grading
}

# adam optimizer is used with a learning rate of 0.001
model.compile(optimizer=Adam(learning_rate=0.001), loss=losses, loss_weights=loss_weights, metrics=['accuracy'])

model.summary() # prints full summary

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

### Training

In [9]:
# Train the model
early_stopping = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

In [10]:
y_present_train.shape, y_grades_train.shape, y_present_val.shape, y_grades_val.shape

((2880,), (2880, 5), (320,), (320, 5))

In [11]:
model.fit(X_train, {'present_output': y_present_train, 'grading_output': y_grades_train}, batch_size=64,
          steps_per_epoch=int(len(X_train) / 64),
          epochs=100,
          validation_data=(X_val, {'present_output': y_present_val, 'grading_output': y_grades_val}),
          callbacks=[early_stopping],
          verbose=1)

Epoch 1/100


2024-12-12 09:57:44.909068: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-12-12 09:57:46.447141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


45/45 [==============================] - ETA: 0s - loss: 2.1231 - present_output_loss: 0.6278 - grading_output_loss: 1.4953 - present_output_accuracy: 0.6660 - grading_output_accuracy: 0.3979

2024-12-12 09:58:00.225066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


45/45 [==============================] - 17s 313ms/step - loss: 2.1231 - present_output_loss: 0.6278 - grading_output_loss: 1.4953 - present_output_accuracy: 0.6660 - grading_output_accuracy: 0.3979 - val_loss: 1.7126 - val_present_output_loss: 0.4866 - val_grading_output_loss: 1.2261 - val_present_output_accuracy: 0.7406 - val_grading_output_accuracy: 0.5531
Epoch 2/100
45/45 [==============================] - 12s 278ms/step - loss: 1.6750 - present_output_loss: 0.4724 - grading_output_loss: 1.2026 - present_output_accuracy: 0.7726 - grading_output_accuracy: 0.5222 - val_loss: 1.6654 - val_present_output_loss: 0.4366 - val_grading_output_loss: 1.2288 - val_present_output_accuracy: 0.8031 - val_grading_output_accuracy: 0.5781
Epoch 3/100
45/45 [==============================] - 13s 281ms/step - loss: 1.5588 - present_output_loss: 0.4361 - grading_output_loss: 1.1227 - present_output_accuracy: 0.8017 - grading_output_accuracy: 0.5500 - val_loss: 1.4726 - val_present_output_loss: 0.4116 

### Validation

In [12]:
# Evaluate the model
scores = model.evaluate(X_val, {'present_output': y_present_val, 'grading_output': y_grades_val})
print("Validation Accuracy (Presence): %.2f%%" % (scores[3] * 100))  # Accuracy for present_output
print("Validation Loss (Presence): %.2f" % scores[1])
print("Validation Accuracy (Grading): %.2f%%" % (scores[4] * 100))  # Accuracy for grading_output
print("Validation Loss (Grading): %.2f" % scores[2])

2024-12-12 10:06:42.565123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 3s 125ms/step - loss: 1.1587 - present_output_loss: 0.3545 - grading_output_loss: 0.8042 - present_output_accuracy: 0.8625 - grading_output_accuracy: 0.6812
Validation Accuracy (Presence): 86.25%
Validation Loss (Presence): 0.35
Validation Accuracy (Grading): 68.12%
Validation Loss (Grading): 0.80


### Testing

In [13]:
# Evaluate the model
scores = model.evaluate(X_test, {'present_output': y_present_test, 'grading_output': y_grades_test})
print("Test Accuracy (Presence): %.2f%%" % (scores[3] * 100))  # Accuracy for present_output
print("Test Loss (Presence): %.2f" % scores[1])
print("Test Accuracy (Grading): %.2f%%" % (scores[4] * 100))  # Accuracy for grading_output
print("Test Loss (Grading): %.2f" % scores[2])

25/25 [==============================] - 3s 123ms/step - loss: 1.4151 - present_output_loss: 0.5067 - grading_output_loss: 0.9084 - present_output_accuracy: 0.8175 - grading_output_accuracy: 0.6787
Test Accuracy (Presence): 81.75%
Test Loss (Presence): 0.51
Test Accuracy (Grading): 67.87%
Test Loss (Grading): 0.91


### Save model

In [14]:
# Save the model
model_json = model.to_json()
with open("../models_TL/ResNet50_pretrained_enhanced.json", "w") as json_file:
    json_file.write(model_json)

# Saving the model and weights
model.save_weights('../models_TL/ResNet50_pretrained_enhanced.weights.h5')